## Loading of Steinmetz data

includes some visualizations

In [1]:
#@title Data retrieval
import os, requests,pdb

fname = []
for j in range(3):
  fname.append('steinmetz_part%d.npz'%j)
url = ["https://osf.io/agvxh/download"]
url.append("https://osf.io/uv3mw/download")
url.append("https://osf.io/ehmw2/download")

for j in range(len(url)):
  if not os.path.isfile(fname[j]):
    try:
      r = requests.get(url[j])
    except requests.ConnectionError:
      print("!!! Failed to download data !!!")
    else:
      if r.status_code != requests.codes.ok:
        print("!!! Failed to download data !!!")
      else:
        with open(fname[j], "wb") as fid:
          fid.write(r.content)


In [2]:
#@title Import matplotlib and set defaults
from matplotlib import rcParams 
from matplotlib import pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
rcParams['figure.figsize'] = [20, 4]
rcParams['font.size'] =15
rcParams['axes.spines.top'] = False
rcParams['axes.spines.right'] = False
rcParams['figure.autolayout'] = True

In [3]:
#@title Data loading
import numpy as np

alldat = np.array([])
for j in range(len(fname)):
  alldat = np.hstack((alldat, np.load('steinmetz_part%d.npz'%j, allow_pickle=True)['dat']))

# select just one of the recordings here. 11 is nice because it has some neurons in vis ctx. 
dat = alldat[11]
print(dat.keys())

dict_keys(['spks', 'wheel', 'pupil', 'response', 'response_time', 'bin_size', 'stim_onset', 'contrast_right', 'contrast_left', 'brain_area', 'feedback_time', 'feedback_type', 'gocue', 'mouse_name', 'date_exp', 'trough_to_peak', 'active_trials', 'contrast_left_passive', 'contrast_right_passive', 'spks_passive', 'pupil_passive', 'wheel_passive', 'prev_reward', 'ccf', 'ccf_axes', 'cellid_orig', 'reaction_time', 'face', 'face_passive', 'licks', 'licks_passive'])


`alldat` contains 39 sessions from 10 mice, data from Steinmetz et al, 2019. Time bins for all measurements are 10ms, starting 500ms before stimulus onset. The mouse had to determine which side has the highest contrast. For each `dat = alldat[k]`, you have the fields below. For extra variables, check out the extra notebook and extra data files (lfp, waveforms and exact spike times, non-binned). 

* `dat['mouse_name']`: mouse name
* `dat['date_exp']`: when a session was performed
* `dat['spks']`: neurons by trials by time bins.    
* `dat['brain_area']`: brain area for each neuron recorded. 
* `dat['ccf']`: Allen Institute brain atlas coordinates for each neuron. 
* `dat['ccf_axes']`: axes names for the Allen CCF. 
* `dat['contrast_right']`: contrast level for the right stimulus, which is always contralateral to the recorded brain areas.
* `dat['contrast_left']`: contrast level for left stimulus. 
* `dat['gocue']`: when the go cue sound was played. 
* `dat['response_times']`: when the response was registered, which has to be after the go cue. The mouse can turn the wheel before the go cue (and nearly always does!), but the stimulus on the screen won't move before the go cue.  
* `dat['response']`: which side the response was (`-1`, `0`, `1`). When the right-side stimulus had higher contrast, the correct choice was `-1`. `0` is a no go response. 
* `dat['feedback_time']`: when feedback was provided. 
* `dat['feedback_type']`: if the feedback was positive (`+1`, reward) or negative (`-1`, white noise burst).  
* `dat['wheel']`: turning speed of the wheel that the mice uses to make a response, sampled at `10ms`. 
* `dat['pupil']`: pupil area  (noisy, because pupil is very small) + pupil horizontal and vertical position.
* `dat['face']`: average face motion energy from a video camera. 
* `dat['licks']`: lick detections, 0 or 1.   
* `dat['trough_to_peak']`: measures the width of the action potential waveform for each neuron. Widths `<=10` samples are "putative fast spiking neurons". 
* `dat['%X%_passive']`: same as above for `X` = {`spks`, `pupil`, `wheel`, `contrast_left`, `contrast_right`} but for  passive trials at the end of the recording when the mouse was no longer engaged and stopped making responses. 
* `dat['prev_reward']`: time of the feedback (reward/white noise) on the previous trial in relation to the current stimulus time. 
* `dat['reaction_time']`: ntrials by 2. First column: reaction time computed from the wheel movement as the first sample above `5` ticks/10ms bin. Second column: direction of the wheel movement (`0` = no move detected).  




In [4]:
def region_finder(region):
    # Find sessions with areas of interest
    session_idx = []
    neuron_idx = []
    for i in range(39):
        dat = alldat[i]
        areas = dat['brain_area']
        decision = areas==region
        if any(decision):
            n = [j for j, x in enumerate(decision) if x]
            session_idx.append(i)
            neuron_idx.append(n)
            #pdb.set_trace()
    return session_idx, neuron_idx

In [ ]:
session_idx, neuron_idx = region_finder("MOs")
dat = alldat[session_idx[1]]
print(dat.keys())

dt = dat['bin_size'] # binning at 10 ms
zebras = dat['spks']
zebras = zebras[neuron_idx[1]]
NT = zebras.shape[-1]
pdb.set_trace()
#ax = plt.subplot(1,5,1)
ax = plt.subplot(1,1,1)
plt.plot(dt * np.arange(NT), 1/dt * dat['spks'][:,response>=0].mean(axis=(0,1)))
#response = dat['response'] # right - nogo - left (-1, 0, 1)
#vis_right = dat['contrast_right'] # 0 - low - high
#vis_left = dat['contrast_left'] # 0 - low - high
#plt.plot(dt * np.arange(NT), 1/dt * dat['spks'][:,response>=0].mean(axis=(0,1))) # left responses
#plt.plot(dt * np.arange(NT), 1/dt * dat['spks'][:,response<0].mean(axis=(0,1))) # right responses
#plt.plot(dt * np.arange(NT), 1/dt * dat['spks'][:,vis_right>0].mean(axis=(0,1))) # stimulus on the right
#plt.plot(dt * np.arange(NT), 1/dt * dat['spks'][:,vis_right==0].mean(axis=(0,1))) # no stimulus on the right

#plt.legend(['left resp', 'right resp', 'right stim', 'no right stim'], fontsize=12)
#ax.set(xlabel  = 'time (sec)', ylabel = 'firing rate (Hz)');

In [5]:
import pdb
dat=alldat[1]
print(dat['contrast_right'])

[1.   0.   0.5  0.   0.   0.   0.   1.   0.   0.   1.   0.5  0.25 1.
 0.25 0.25 0.25 0.5  0.   0.   0.   1.   1.   0.5  0.   0.5  0.   0.
 0.   0.   1.   0.25 1.   0.   0.   0.   0.25 0.   0.   0.   0.   0.25
 0.   1.   0.5  0.25 0.   0.   0.   0.   0.   0.   0.   0.5  1.   0.5
 0.5  0.   0.   0.   1.   0.   0.   0.25 0.25 1.   0.   1.   0.   0.
 0.   0.   1.   0.   1.   0.25 0.25 0.   0.   0.5  0.5  0.   0.5  1.
 0.   0.25 0.   0.   0.   0.   0.25 1.   1.   0.25 0.   0.   0.   0.25
 0.25 0.   0.5  0.5  1.   1.   0.   0.   0.   0.   0.   0.   1.   0.25
 0.5  0.   0.   0.25 0.   0.   0.   0.   0.   0.   0.   0.5  0.   0.
 0.   0.   1.   0.   0.5  0.25 0.25 0.25 0.25 0.25 0.25 0.   1.   1.
 1.   0.   0.   0.5  0.5  0.   0.   0.   0.   1.   1.   1.   1.   1.
 1.   1.   1.   1.   1.   1.   1.   1.   1.   1.   0.   1.   0.5  0.
 0.   0.   0.   0.   0.   1.   0.   0.   1.   0.   0.5  1.   1.   0.5
 0.25 0.25 0.25 0.25 0.   0.5  0.5  0.5  0.5  0.25 0.25 0.5  1.   0.5
 0.   0.   0.   0.   0.25

In [6]:
def what_side(dataset,contrast_choice):
    saver=[]
    if contrast_choice == 'Contralateral':
        word='contrast_right'
        other='contrast_left'
    else:
        word='contrast_left'
        other='contrast_right'
    for i in range(len(dataset[word])):
        if dataset[word][i] > dataset[other][i]
        #dataset[word][i] >0. and dataset[other][i] >0.:
            #pdb.set_trace()
            saver.append(i)
    return saver
    

SyntaxError: invalid syntax (<ipython-input-6-a64fb4ff6561>, line 10)

In [ ]:
what_side(dat,'Ipsilateral')

In [18]:
#@title basic plots of population average
from matplotlib import pyplot as plt
dt = dat['bin_size'] # binning at 10 ms
NT = dat['spks'].shape[-1]

ax = plt.subplot(1,5,1)
response = dat['response'] # right - nogo - left (-1, 0, 1)
vis_right = dat['contrast_right'] # 0 - low - high
vis_left = dat['contrast_left'] # 0 - low - high
plt.plot(dt * np.arange(NT), 1/dt * dat['spks'][:,response>=0].mean(axis=(0,1))) # left responses
plt.plot(dt * np.arange(NT), 1/dt * dat['spks'][:,response<0].mean(axis=(0,1))) # right responses
plt.plot(dt * np.arange(NT), 1/dt * dat['spks'][:,vis_right>0].mean(axis=(0,1))) # stimulus on the right
plt.plot(dt * np.arange(NT), 1/dt * dat['spks'][:,vis_right==0].mean(axis=(0,1))) # no stimulus on the right

plt.legend(['left resp', 'right resp', 'right stim', 'no right stim'], fontsize=12)
ax.set(xlabel  = 'time (sec)', ylabel = 'firing rate (Hz)');

C:\Users\Andrew\.conda\envs\nma\lib\site-packages\IPython\core\events.py:89: UserWarning: tight_layout not applied: number of columns in subplot specifications must be multiples of one another.
  func(*args, **kwargs)


In [ ]:
# groupings of brain regions
regions = ["vis ctx", "thal", "hipp", "other ctx", "midbrain", "basal ganglia", "cortical subplate", "other"]
brain_groups = [["VISa", "VISam", "VISl", "VISp", "VISpm", "VISrl"], # visual cortex
                ["CL", "LD", "LGd", "LH", "LP", "MD", "MG", "PO", "POL", "PT", "RT", "SPF", "TH", "VAL", "VPL", "VPM"], # thalamus
                ["CA", "CA1", "CA2", "CA3", "DG", "SUB", "POST"], # hippocampal
                ["ACA", "AUD", "COA", "DP", "ILA", "MOp", "MOs", "OLF", "ORB", "ORBm", "PIR", "PL", "SSp", "SSs", "RSP"," TT"], # non-visual cortex
                ["APN", "IC", "MB", "MRN", "NB", "PAG", "RN", "SCs", "SCm", "SCig", "SCsg", "ZI"], # midbrain
                ["ACB", "CP", "GPe", "LS", "LSc", "LSr", "MS", "OT", "SNr", "SI"], # basal ganglia 
                ["BLA", "BMA", "EP", "EPd", "MEA"] # cortical subplate
                ]

nareas = 4 # only the top 4 regions are in this particular mouse
NN = len(dat['brain_area']) # number of neurons
barea = nareas * np.ones(NN, ) # last one is "other"
for j in range(nareas):
  barea[np.isin(dat['brain_area'], brain_groups[j])] = j # assign a number to each region

In [ ]:
#Below is the code for the project
#
#
#
#
#

In [ ]:
def get_region_data(region):
    session_idx ,neuron_idx = region_finder(region) 

    region_spiketimes = {} #create a dictionary with {}
    region_lfp = {}
    for is_session in session_idx:
        region_neurons = np.where(alldat[is_session]["brain_area"] == region)[0]
        region_spiketimes[is_session] = {}
        #region_lfp[is_session] = {}
        for neuron in region_neurons: 
                                 #visual_FR_test[(is_session,neuron)]=alldat[is_session]["spks"][neuron,:,:]
            region_spiketimes[is_session][neuron]=alldat[is_session]["spks"][neuron,:,:] 
            #region_lfp[is_session][neuron]=alldat[is_session]["spks"][neuron,:,:] 
            #rgion_spiketimes first key is session number (only included are those with selected brain region). 
            # Then neuron number from that brain region.
 
    return region_spiketimes

region = "VISp" #primary visual cortex
region_spiketimes = get_region_data(region)

In [11]:
[session_id,neuron_id]=region_finder("VISp")
#dataset=alldat[session_id]
#[new_ids]=what_side(dataset,'Contralateral')
#session_idx ,neuron_idx = region_finder(region)
storer=[]
bins=0.01
test=session_id[1]
word='contrast_right'
other='contrast_left'
#for i in session_id:
    #dat=alldat[test]
    #barea=dat['brain_area']
    #datter=dat['spks'][barea=='VISp']
    #trial_FR = np.sum(datter, axis=0) / (np.shape(dat['spks'])[2] * bins / 1000)
    #norm_FR = trial_FR/ np.mean(trial_FR) #array of normalized FR for each neuron in area on L or R choice trials (n_neurons x n_trials)
    #print(norm_FR)
    #pdb.set_trace()
    #storer.append(norm_FR)
dat=alldat[test]
barea=dat['brain_area']
datter=dat['spks'][barea=='VISp']
#pdb.set_trace()

datter=datter[:,np.logical_and(dat[word]>dat[other],dat['feedback_type']>0)]
#datter=datter[:,dat['feedback_type']>0]

pdb.set_trace()
#if dataset[word][i] >0. and dataset[other][i] ==0.:

trial_FR = np.sum(datter, axis=2) / (np.shape(dat['spks'])[2] * bins / 1000)
norm_FR = trial_FR/ np.mean(trial_FR) #array of normalized FR for each neuron in area on L or R choice trials (n_neurons x n_trials)
#print(norm_FR)
#pdb.set_trace()
#storer.append(norm_FR)


#turn dat['spks'] into [n_neurons, n_trials] array of normalized firing rates on each trial
#bins = 0.01 #10ms bins
#trial_FR = np.sum(dat['spks'], axis=2) / (np.shape(dat['spks'])[2] * bins / 1000)
#norm_FR = trial_FR[:,LorR] / np.mean(trial_FR) #array of normalized FR for each neuron in area on L or R choice trials (n_neurons x n_trials)
    
#print(neuron_id)

--Return--
None
> <ipython-input-11-6f11b2351bc1>(27)<module>()
     25 #datter=datter[:,dat['feedback_type']>0]
     26 
---> 27 pdb.set_trace()
     28 #if dataset[word][i] >0. and dataset[other][i] ==0.:
     29 

ipdb> np.logical_and(dat[word]>dat[other],dat['feedback_type']>0)
array([False, False,  True,  True,  True, False, False,  True,  True,
       False, False,  True,  True,  True, False,  True,  True,  True,
       False,  True, False,  True, False, False, False, False, False,
       False, False,  True, False, False, False, False, False, False,
        True, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False,  True,
        True,  True, False,  True, False, False, False, False,  True,
       False,  True, False, False, False, False,  True, False, False,
       False,  True,  True, False, False, False, False,  True,  True,
       False,  True, False, False, False, False,  True, False, False,
       False, Fal

BdbQuit: 

In [23]:
#import pypi
def ismember(A, B):
    return [ np.sum(a == B) for a in A ]

ModuleNotFoundError: No module named 'pypi'

In [25]:
[session_id,neuron_id]=region_finder("VISp")
[session_idone,neuron_idone]=region_finder("PL")
[session_idtwo,neuron_idtwo]=region_finder("MOs")
[session_idthree,neuron_idthree]=region_finder("CP")
[session_idfour,neuron_idfour]=region_finder("SNr")
[session_idfive,neuron_idfive]=region_finder("ZI")
[session_idsix,neuron_idsix]=region_finder("SCm")

newone=[]
newone.append(session_idone)
newone.append(session_idtwo)
newone.append(session_idthree)
newone.append(session_idfour)
newone.append(session_idfive)
newone.append(session_idsix)


newone=sum(newone,[])
print(session_id)
print(newone)

#ismember(session_id,newone)
apple=list(set(session_id) & set(newone))
print(apple)


[0, 2, 3, 7, 9, 11, 13, 19, 21, 24, 25, 38]
[4, 7, 8, 11, 12, 21, 24, 26, 35, 38, 0, 3, 4, 7, 11, 12, 13, 21, 24, 25, 26, 29, 30, 31, 34, 35, 36, 38, 6, 10, 17, 28, 32, 17, 30, 32, 37, 12, 14, 17, 32, 9, 12, 13, 30, 31, 35, 36]
[0, 3, 38, 7, 9, 11, 13, 21, 24, 25]


In [14]:
[session_id,neuron_id]=region_finder("VISp")
[session_idone,neuron_idone]=region_finder("PL")
[session_idtwo,neuron_idtwo]=region_finder("MOs")
[session_idthree,neuron_idthree]=region_finder("CP")
[session_idfour,neuron_idfour]=region_finder("SNr")
[session_idfive,neuron_idfive]=region_finder("ZI")
[session_idsix,neuron_idsix]=region_finder("SCm")

newone=[]
newone.append(session_idone)
newone.append(session_idtwo)
newone.append(session_idthree)
newone.append(session_idfour)
newone.append(session_idfive)
newone.append(session_idsix)


newone=sum(newone,[])
print(session_id)
print(newone)
apple=list(set(session_id) & set(newone))
print(apple)


#pdb.set_trace()
#dataset=alldat[session_id]
#[new_ids]=what_side(dataset,'Contralateral')
#session_idx ,neuron_idx = region_finder(region)
storer=[]
bins=0.01
test=session_id[1]
word='contrast_right'
other='contrast_left'
#for i in session_id:
    #dat=alldat[test]
    #barea=dat['brain_area']
    #datter=dat['spks'][barea=='VISp']
    #trial_FR = np.sum(datter, axis=0) / (np.shape(dat['spks'])[2] * bins / 1000)
    #norm_FR = trial_FR/ np.mean(trial_FR) #array of normalized FR for each neuron in area on L or R choice trials (n_neurons x n_trials)
    #print(norm_FR)
    #pdb.set_trace()
    #storer.append(norm_FR)
dat=alldat[test]
barea=dat['brain_area']
datter=dat['spks'][barea=='VISp']
#pdb.set_trace()

datter=datter[:,np.logical_and(dat[word]>dat[other],dat['feedback_type']>0)]
#datter=datter[:,dat['feedback_type']>0]

pdb.set_trace()
#if dataset[word][i] >0. and dataset[other][i] ==0.:

trial_FR = np.sum(datter, axis=2) / (np.shape(dat['spks'])[2] * bins / 1000)
norm_FR = trial_FR/ np.mean(trial_FR) #array of normalized FR for each neuron in area on L or R choice trials (n_neurons x n_trials)
#print(norm_FR)
#pdb.set_trace()
#storer.append(norm_FR)


#turn dat['spks'] into [n_neurons, n_trials] array of normalized firing rates on each trial
#bins = 0.01 #10ms bins
#trial_FR = np.sum(dat['spks'], axis=2) / (np.shape(dat['spks'])[2] * bins / 1000)
#norm_FR = trial_FR[:,LorR] / np.mean(trial_FR) #array of normalized FR for each neuron in area on L or R choice trials (n_neurons x n_trials)
    
#print(neuron_id)

None
> <ipython-input-14-45359305a8d2>(17)<module>()
     15 for i in range(len(session_id[:])):
     16     pdb.set_trace()
---> 17     if any(session_id[i]==session_idone):
     18         holdon.append(session_id[i])
     19 

ipdb> i
0
ipdb> session_idone
[4, 7, 8, 11, 12, 21, 24, 26, 35, 38, [0, 3, 4, 7, 11, 12, 13, 21, 24, 25, 26, 29, 30, 31, 34, 35, 36, 38], [6, 10, 17, 28, 32], [17, 30, 32, 37], [12, 14, 17, 32], [9, 12, 13, 30, 31, 35, 36]]
ipdb> session_id
[0, 2, 3, 7, 9, 11, 13, 19, 21, 24, 25, 38]
ipdb> session_id[i]
0
ipdb> session_id[i]==session_idone
False
ipdb> session_id[3]==session_idone
False
ipdb> session_idtwo
[0, 3, 4, 7, 11, 12, 13, 21, 24, 25, 26, 29, 30, 31, 34, 35, 36, 38]
ipdb> type(session_idtwo)
<class 'list'>
ipdb> exit


BdbQuit: 

In [34]:
plt.plot(range(214),)

In [46]:
print(np.shape(norm_FR))
print(np.shape(dat))

(228, 250)
()


In [22]:

def getChoiceCells(dat):
    #make array of choices. left choices = 1, right choices = 0, ignore no-go trials
    LorR = dat['response']!=0 
    choices = dat['response'][LorR]
    choices[choices == -1] = 0
    
    #turn dat['spks'] into [n_neurons, n_trials] array of normalized firing rates on each trial
    bins = 0.01 #10ms bins
    trial_FR = np.sum(dat['spks'], axis=2) / (np.shape(dat['spks'])[2] * bins / 1000)
    norm_FR = trial_FR[:,LorR] / np.mean(trial_FR) #array of normalized FR for each neuron in area on L or R choice trials (n_neurons x n_trials)
    
    # Define logistic regression model
    log_reg = LogisticRegression(penalty="none").fit(norm_FR.T, choices)
    y_pred = log_reg.predict(norm_FR.T)
    accuracy = (choices == y_pred).mean()
    
    most_predictive = np.quantile(log_reg.coef_, [0.10, 0.90])
    ispi_cells = log_reg.coef_ > most_predictive[1]
    contra_cells = log_reg.coef_ < most_predictive[0]
    
    return ispi_cells, contra_cells

def plot_weights(models, sharey=True):
    """Draw a stem plot of weights for each model in models dict."""
    n = len(models)
    f = plt.figure(figsize=(10, 2.5 * n))
    axs = f.subplots(n, sharex=True, sharey=sharey)
    axs = np.atleast_1d(axs)

    for ax, (title, model) in zip(axs, models.items()):

        ax.margins(x=.02)
        stem = ax.stem(model.coef_.squeeze(), use_line_collection=True)
        stem[0].set_marker(".")
        stem[0].set_color(".2")
        stem[1].set_linewidths(.5)
        stem[1].set_color(".2")
        stem[2].set_visible(False)
        ax.axhline(0, color="C3", lw=3)
        ax.set(ylabel="Weight", title=title)
    ax.set(xlabel="Neuron (a.k.a. feature)")
    f.tight_layout()

In [ ]:
# to_remove solution


def make_design_matrix(stim, d=25):
  """Create time-lag design matrix from stimulus intensity vector.

  Args:
    stim (1D array): Stimulus intensity at each time point.
    d (number): Number of time lags to use.

  Returns
    X (2D array): GLM design matrix with shape T, d

  """
  # Create version of stimulus vector with zeros before onset
  padded_stim = np.concatenate([np.zeros(d - 1), stim])

  # Construct a matrix where each row has the d frames of
  # the stimulus proceeding and including timepoint t
  T = len(stim)  # Total number of timepoints (hint: number of stimulus frames)
  X = np.zeros((T, d))
  for t in range(T):
      X[t] = padded_stim[t:t + d]

  return X


# Make design matrix
X = make_design_matrix(stim)

# Visualize
with plt.xkcd():
  plot_glm_matrices(X, spikes, nt=50)

In [20]:
#@title plots by brain region and visual conditions
for j in range(nareas):
  ax = plt.subplot(1,nareas,j+1)
  
  plt.plot(1/dt *  dat['spks'][barea==j][:,np.logical_and(vis_left==0, vis_right>0)].mean(axis=(0,1)))
  plt.plot(1/dt *  dat['spks'][barea==j][:,np.logical_and(vis_left>0 , vis_right==0)].mean(axis=(0,1)))
  plt.plot(1/dt *  dat['spks'][barea==j][:,np.logical_and(vis_left==0 , vis_right==0)].mean(axis=(0,1)))
  plt.plot(1/dt *  dat['spks'][barea==j][:,np.logical_and(vis_left>0, vis_right>0)].mean(axis=(0,1)))  
  plt.text(.25, .92, 'n=%d'%np.sum(barea==j), transform=ax.transAxes)
 
  if j==0:
    plt.legend(['right only', 'left only', 'neither', 'both'], fontsize=12)
  ax.set(xlabel = 'binned time', ylabel = 'mean firing rate (Hz)', title = regions[j])


In [7]:
#@title plots by brain region and response type
for j in range(nareas):
  ax = plt.subplot(1,nareas,j+1)
  plt.title(regions[j])
  if np.sum(barea==j)==0:
    continue
  plt.plot(1/dt * dat['spks'][barea==j][:,response<0].mean(axis=(0,1)))  
  plt.plot(1/dt * dat['spks'][barea==j][:,response>0].mean(axis=(0,1)))
  plt.plot(1/dt * dat['spks'][barea==j][:,response==0].mean(axis=(0,1)))
 
  if j==0:
    plt.legend(['resp = left', 'resp = right', 'resp = none'], fontsize=12)
  ax.set(xlabel = 'time', ylabel = 'mean firing rate (Hz)')


In [8]:
#@title top PC directions from stimulus + response period, with projections of the entire duration
from sklearn.decomposition import PCA 

droll = np.reshape(dat['spks'][:,:,51:130], (NN,-1)) # first 80 bins = 1.6 sec
droll = droll - np.mean(droll, axis=1)[:, np.newaxis]
model = PCA(n_components = 5).fit(droll.T)
W = model.components_
pc_10ms = W @ np.reshape(dat['spks'], (NN,-1))
pc_10ms = np.reshape(pc_10ms, (5, -1, NT))

In [9]:
#@title The top PCs capture most variance across the brain. What do they care about? 
plt.figure(figsize= (20, 6))
for j in range(len(pc_10ms)):
  ax = plt.subplot(2,len(pc_10ms)+1,j+1)
  pc1 = pc_10ms[j]

  plt.plot(pc1[np.logical_and(vis_left==0, vis_right>0), :].mean(axis=0))  
  plt.plot(pc1[np.logical_and(vis_left>0, vis_right==0), :].mean(axis=0))
  plt.plot(pc1[np.logical_and(vis_left==0, vis_right==0), :].mean(axis=0))
  plt.plot(pc1[np.logical_and(vis_left>0, vis_right>0), :].mean(axis=0))
   
  if j==0:
    plt.legend(['right only', 'left only', 'neither', 'both'], fontsize=8)
  ax.set(xlabel = 'binned time', ylabel = 'mean firing rate (Hz)')
  plt.title('PC %d'%j)

  ax = plt.subplot(2,len(pc_10ms)+1,len(pc_10ms)+1 + j+1)
  
  plt.plot(pc1[response>0, :].mean(axis=0))  
  plt.plot(pc1[response<0, :].mean(axis=0))
  plt.plot(pc1[response==0, :].mean(axis=0))

  if j==0:
    plt.legend(['resp = left', 'resp = right', 'resp = none'], fontsize=8)
  ax.set(xlabel = 'binned time', ylabel = 'mean firing rate (Hz)')
  plt.title('PC %d'%j)

In [10]:
#@title now sort all trials by response latency and see if the PCs care about that.
from scipy.stats import zscore

isort = np.argsort(dat['response_time'].flatten())

for j in range(len(pc_10ms)):
  ax = plt.subplot(1,len(pc_10ms)+1,j+1)
  pc1 = zscore(pc_10ms[j])
  plt.imshow(pc1[isort, :], aspect='auto', vmax=2, vmin = -2, cmap = 'gray')
  ax.set(xlabel = 'binned time', ylabel = 'trials sorted by latency')
  plt.title('PC %d'%j)

In [11]:
#@title correct vs incorrect trials
# the following are the correct responses:
# if vis_left > vis_right : response >0
# if vis_left < vis_right : response <0
# if vis_left = vis_right : response =0
# trials below red line are incorrect
is_correct = np.sign(response)==np.sign(vis_left-vis_right)

# sort by correct, and then by response
isort = np.argsort(-is_correct.astype('float32') + response/10) 

nwrong = np.sum(is_correct)
for j in range(len(pc_10ms)):
  ax = plt.subplot(1,len(pc_10ms)+1,j+1)
  pc1 = zscore(pc_10ms[j])
  plt.imshow(pc1[isort, :], aspect='auto', vmax=2, vmin = -2, cmap = 'gray')
  ax.set(xlabel = 'binned time')
  if j==0:
    ax.set(ylabel = 'trials sorted by latency')  
  plt.title('PC %d'%j)

  plt.plot([0, NT], [nwrong, nwrong], 'r')


In [12]:
# plot the behavioral data (pupil area is noisy because it's very small)

ax = plt.subplot(1,5,1)
plt.plot(dat['pupil'][0, :].mean(0));
ax.set(ylabel='pupil area', xlabel = 'binned time', title='Pupil dynamics')

yl = [-10, 10]
ax = plt.subplot(1,5,2)
plt.plot(dat['wheel'][0, response>0].mean(0));
ax.set(ylim=yl)
ax.set(ylim=yl, ylabel='wheel turns', xlabel = 'binned time', title='Left choices');

ax = plt.subplot(1,5,3)
plt.plot(dat['wheel'][0, response<0].mean(0));
ax.set(ylim=yl)
ax.set(ylim=yl, ylabel='wheel turns', xlabel = 'binned time', title='Right choices');

ax = plt.subplot(1,5,4)
plt.plot(dat['wheel'][0, response==0].mean(0));
ax.set(ylim=yl, ylabel='wheel turns', xlabel = 'binned time', title='No go choices');


In [13]:
# plots by brain region and visual conditions for PASSIVE trials
vis_left_p = dat['contrast_left_passive']
vis_right_p = dat['contrast_right_passive']
for j in range(nareas):
  ax = plt.subplot(1,nareas,j+1)
  plt.title(regions[j])
  
  plt.plot(1/dt *  dat['spks_passive'][barea==j][:,np.logical_and(vis_left_p==0, vis_right_p>0)].mean(axis=(0,1)))
  plt.plot(1/dt *  dat['spks_passive'][barea==j][:,np.logical_and(vis_left_p>0 , vis_right_p==0)].mean(axis=(0,1)))
  #plt.plot(1/dt *  dat['spks_passive'][barea==j][:,np.logical_and(vis_left_p==0 , vis_right_p==0)].mean(axis=(0,1)))
  plt.plot(1/dt *  dat['spks_passive'][barea==j][:,np.logical_and(vis_left_p>0, vis_right_p>0)].mean(axis=(0,1)))  
  plt.text(.25, .92, 'n=%d'%np.sum(barea==j), transform=ax.transAxes)
 
  if j==0:
    plt.legend(['right only', 'left only', 'both'], fontsize=12)
  ax.set(xlabel = 'binned time', ylabel = 'mean firing rate (Hz)')

In [14]:
# for more variables check out the additional notebook (load_steinmetz_extra) which includes LFP, waveform shapes and exact spike times (non-binned)